In [63]:
# packages
import numpy as np
import pandas as pd
import pyblp

In [71]:
# import csv
path = "/Users/fernramoutar/Dropbox/classes/year-2/io-2/ps1/"
motorcycles = pd.read_csv(path+'psetOne.csv',sep = ',')

In [72]:
# rename variables
motorcycles .rename(columns = {'Price':'prices', 'Market':"market_ids",'z1':'demand_instruments0','z2':'demand_instruments1','z3':'demand_instruments2','z4':'demand_instruments3'}, inplace = True)

# set up for brand FEs
motorcycles ['Brand'] = 1
motorcycles .loc[df['Brand2']==1, 'Brand'] = 2
motorcycles .loc[df['Brand3']==3, 'Brand'] = 3

motorcycles 

,market_ids,Constant,prices,EngineSize,SportsBike,Brand2,Brand3,demand_instruments0,demand_instruments1,demand_instruments2,demand_instruments3,shares,Brand
0,1,1,1.554708,3.0,1,0,0,0.237685,0.063116,0.034361,0.056459,0.239077,1
1,1,1,1.559811,5.0,1,0,0,0.002033,0.186073,0.326445,0.062091,0.072184,1
2,1,1,1.857186,3.0,1,1,0,0.164550,0.033308,0.075316,0.278736,0.000460,2
3,1,1,1.995438,5.0,1,0,1,0.485538,0.117025,0.345850,0.084891,0.587867,1
4,1,1,4.017987,7.5,1,0,1,0.582503,0.368032,0.652134,0.509693,0.033746,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,150,1,2.010710,3.0,1,1,0,0.257544,0.068546,0.095527,0.185333,0.000548,2
1043,150,1,2.001591,7.5,0,1,0,0.042531,0.474048,0.285886,0.263406,0.024166,2
1044,150,1,1.692977,5.0,1,0,1,0.021038,0.453095,0.079379,0.276368,0.355868,1
1045,150,1,2.337965,7.5,1,0,1,0.565742,0.190674,0.695795,0.044509,0.347937,1


In [73]:
# define formulation for linear demand model
linear_formulation = pyblp.Formulation('1 + prices + EngineSize + SportsBike', absorb = 'C(Brand)') # should we do fixed effects?

# define formulation for non-linear demand model
non_linear_formulation = pyblp.Formulation('0 + prices + EngineSize') 

# combine
product_formulations = (linear_formulation, non_linear_formulation)

In [74]:
# monte carlo draws
mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})
mc_integration

Configured to construct nodes and weights with Monte Carlo simulation with options {seed: 0}.

In [76]:
# problem
mc_problem = pyblp.Problem(product_formulations, motorcycles , integration=mc_integration)
mc_problem

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     I     K1    K2    MD    ED 
---  ----  ----  ----  ----  ----  ----
150  1047  7500   3     2     6     1  

Formulations:
       Column Indices:           0         1           2     
-----------------------------  ------  ----------  ----------
 X1: Linear Characteristics    prices  EngineSize  SportsBike
X2: Nonlinear Characteristics  prices  EngineSize            


Dimensions:
 T    N     I     K1    K2    MD    ED 
---  ----  ----  ----  ----  ----  ----
150  1047  7500   3     2     6     1  

Formulations:
       Column Indices:           0         1           2     
-----------------------------  ------  ----------  ----------
 X1: Linear Characteristics    prices  EngineSize  SportsBike
X2: Nonlinear Characteristics  prices  EngineSize            

In [77]:
# solve
results = mc_problem.solve(sigma=np.ones((2, 2)), optimization=bfgs)
results

Solving the problem ...

Nonlinear Coefficient Initial Values:
  Sigma:       prices       EngineSize    |  Sigma Squared:     prices       EngineSize  
----------  -------------  -------------  |  --------------  -------------  -------------
  prices    +1.000000E+00                 |      prices      +1.000000E+00  +1.000000E+00
EngineSize  +1.000000E+00  +1.000000E+00  |    EngineSize    +1.000000E+00  +2.000000E+00
Starting optimization ...


At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
Encountered a numerical error when computing delta. This problem is often due to prior problems, overflow, or nonpositive shares, and can sometimes be mitigated by choosing smaller initial parameter values, setting more 


At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
Encountered a numerical error when computing delta. This problem is often due to prior problems, overflow, or nonpositive shares, and can sometimes be mitigated by choosing smaller initial parameter values, setting more conservative bounds on parameters or shares, rescaling data, removing outliers, changing the floating point precision, or using different optimization, iteration, or integration configurations. Errors encountered: divide by zero.

 1         5             8          7255         21918        0     +6.466003E-02  +8.419533E-02  +8.286858E-01  +3.984996E-01, +2.217304E-01, -3.080111E-01

At least one error was encountered. As long as the optimizati


At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
Encountered a numerical error when computing delta. This problem is often due to prior problems, overflow, or nonpositive shares, and can sometimes be mitigated by choosing smaller initial parameter values, setting more conservative bounds on parameters or shares, rescaling data, removing outliers, changing the floating point precision, or using different optimization, iteration, or integration configurations. Errors encountered: divide by zero.

 1         13           17          7746         23404        0     +4.026369E-03  +2.735034E-03  +1.900466E-01  +5.096920E-01, +9.326934E-02, -4.181837E-01

At least one error was encountered. As long as the optimizati


At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
Encountered a numerical error when computing delta. This problem is often due to prior problems, overflow, or nonpositive shares, and can sometimes be mitigated by choosing smaller initial parameter values, setting more conservative bounds on parameters or shares, rescaling data, removing outliers, changing the floating point precision, or using different optimization, iteration, or integration configurations. Errors encountered: divide by zero.
The fixed point computation of delta failed to converge. This problem can sometimes be mitigated by increasing the maximum number of fixed point iterations, increasing the fixed point tolerance, choosing more reasonable 


Encountered a numerical error when computing delta. This problem is often due to prior problems, overflow, or nonpositive shares, and can sometimes be mitigated by choosing smaller initial parameter values, setting more conservative bounds on parameters or shares, rescaling data, removing outliers, changing the floating point precision, or using different optimization, iteration, or integration configurations. Errors encountered: divide by zero.
The fixed point computation of delta failed to converge. This problem can sometimes be mitigated by increasing the maximum number of fixed point iterations, increasing the fixed point tolerance, choosing more reasonable initial parameter values, setting more conservative parameter or share bounds, or using different iteration or optimization configurations.

Computed results after 00:00:15.

Problem Results Summary:
GMM     Objective      Gradient         Hessian         Hessian     Clipped  Weighting Matrix
Step      Value          Norm      

Problem Results Summary:
GMM     Objective      Gradient         Hessian         Hessian     Clipped  Weighting Matrix  Covariance Matrix
Step      Value          Norm       Min Eigenvalue  Max Eigenvalue  Shares   Condition Number  Condition Number 
----  -------------  -------------  --------------  --------------  -------  ----------------  -----------------
 2    +3.264694E-11  +2.248370E-05  -6.819015E-02   +1.832238E+01      0      +1.828735E+02      +3.253655E+05  

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:01:29       Yes          26           33         271803       820521   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
  Sigma:        prices         EngineSize     |  Sigma Squared:      prices         EngineSize   
----------  ---------------  ------